In [1]:
import Cleaner
import EmpiricDistribution
import plotly.express as px
import warnings
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

cleaner = Cleaner.Cleaner()

cleaner.set_path_info({'7': '07_v2.xlsx', '50':'50.xlsx'})

df = cleaner.process_data(export_csv=True)

df

,inicio,final,Usuarios,hora_salida,fecha,unidad,timestamp,siguiente_salida,siguiente_inicio,delta_tiempo,minutos_viaje,trayecto
0,A2,GLAXO,5.0,06:24,2025-02-27,50,2025-02-27 06:24:00,2025-02-27 06:36:00,GLAXO,0 days 00:12:00,12.0,A2-GLAXO
1,GLAXO,DISNEY,0.0,06:36,2025-02-27,50,2025-02-27 06:36:00,2025-02-27 06:45:00,DISNEY,0 days 00:09:00,9.0,GLAXO-DISNEY
2,DISNEY,A2,3.0,06:45,2025-02-27,50,2025-02-27 06:45:00,2025-02-27 06:59:00,A2,0 days 00:14:00,14.0,DISNEY-A2
3,A2,GLAXO,7.0,06:59,2025-02-27,50,2025-02-27 06:59:00,2025-02-27 07:10:00,GLAXO,0 days 00:11:00,11.0,A2-GLAXO
4,GLAXO,DISNEY,0.0,07:10,2025-02-27,50,2025-02-27 07:10:00,2025-02-27 07:18:00,DISNEY,0 days 00:08:00,8.0,GLAXO-DISNEY
...,...,...,...,...,...,...,...,...,...,...,...,...
33,GLAXO,DISNEY,2.0,14:47,2025-08-15,50,2025-08-15 14:47:00,2025-08-15 14:58:00,DISNEY,0 days 00:11:00,11.0,GLAXO-DISNEY
34,DISNEY,A2,3.0,14:58,2025-08-15,50,2025-08-15 14:58:00,2025-08-15 15:19:00,A2,0 days 00:21:00,21.0,DISNEY-A2
35,A2,GLAXO,11.0,15:19,2025-08-15,50,2025-08-15 15:19:00,2025-08-15 15:28:00,GLAXO,0 days 00:09:00,9.0,A2-GLAXO
36,GLAXO,DISNEY,2.0,15:28,2025-08-15,50,2025-08-15 15:28:00,2025-08-15 15:39:00,DISNEY,0 days 00:11:00,11.0,GLAXO-DISNEY


In [2]:
empiric = EmpiricDistribution.EmpiricDistribution()

empiric.set_histogram(df['minutos_viaje'], n_bins=15)

empiric.train(show_histogram=True, show_process=True)

Area calculada: 3788.633333333334
----------------------------------------- 
Ecuaciones (Sin escalar): 
 ----------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
samples = empiric.generate_samples(n_samples=1000)

0.21196650347122492
0.82717569069767
0.08114222188791698
0.4590318849744235
0.13020022754137317
0.31262965538221
0.29492342016098905
0.6470343925378548
0.5131534677308127
0.04247818896544053
0.8834097042195266
0.5829463352595308
0.1724617008545064
0.1434574571370656
0.5602596158886889
0.97090644145085
0.05071183991827333
0.6858024747686062
0.2717483994115917
0.19852249312170245
0.63583732626384
0.6266629922700916
0.6959432225504253
0.8194051681892496
0.5317483362320544
0.14493298499719554
0.14222499097841168
0.5847637177513213
0.8432273311517039
0.4051630720266485
0.20927646040344294
0.9762882629430842
0.5464243363672547
0.6308555216625369
0.46343108766155805
0.34582215776326775
0.10339674929280696
0.45351380833770494
0.7870914268904494
0.1728571710438641
0.6596425215642203
0.41904903791700177
0.2542438431115538
0.4303752653907975
0.03570182655513332
0.8787990409646967
0.8714431344462794
0.49434178845305354
0.8707755145892444
0.7763299806372608
0.3585453552922856
0.5025293148396084
0.7

In [9]:
px.histogram(samples)

In [5]:
stats = df.groupby(['trayecto'])['minutos_viaje'].agg(['mean', 'std', 'count', 'min', 'max'])
stats

,mean,std,count,min,max
trayecto,,,,,
A2-GLAXO,7.991935,2.238159,744,4.0,28.0
DISNEY-A2,14.780282,3.195052,710,2.0,28.0
GLAXO-DISNEY,11.112772,3.838516,736,2.0,26.0


In [6]:


counts, bin_edges = np.histogram(df['minutos_viaje'], bins=20)


counts

counts

array([ 14,  12,  39, 402, 243, 200, 334, 145, 138, 146, 261,  77,  53,
        81,  17,  11,   9,   4,   1,   3])

In [7]:
fig = px.histogram(df, x='minutos_viaje')
fig.update_layout(title = f'Distribucion de tiempo entre inicios de viaje', title_x = 0.5)
fig.show()

In [8]:
trayectos = ['A2-GLAXO', 'GLAXO-DISNEY', 'DISNEY-A2']
time_intervals = [('07:00', '09:30'), ('13:30', '18:00'), ('06:00', '07:00'), ('09:30', '13:30'), ('18:30', '22:15')]
for trayecto in trayectos:
    sub_trayecto = df.loc[df['trayecto'] == trayecto]
    for time in time_intervals:
        sub_time = sub_trayecto.loc[(sub_trayecto['hora_salida'] >= time[0]) & (sub_trayecto['hora_salida'] < time[1])]
        fig = px.histogram(sub_time, x='minutos_viaje', nbins=12)
        fig.update_layout(title = f'Distribucion de tiempo entre inicios de viaje en el trayecto {trayecto} en el intervalo {time}', title_x = 0.5)
        fig.show()